# Ethereum Fraud Detection

## Import Packages and Read Data

In [4]:
import pandas as pd
import numpy as np

In [5]:
df_train = pd.read_csv('train_accounts.csv')
df_test = pd.read_csv('test_accounts.csv')
df_transactions = pd.read_csv('transactions.csv')

In [6]:
train_list = df_train.account.to_list()
test_list = df_test.account.to_list()
whole_list = train_list + test_list

print('Training records:', len(train_list), 'Testing records', len(test_list))
print('Total records:', len(whole_list), len(set(whole_list)))

fraud_list = df_train[df_train.flag == 1].account.to_list()
print('Fraud records:', len(fraud_list))

Training records: 25198 Testing records 6300
Total records: 31498 31498
Fraud records: 2455


In [7]:
# Flag Distribution
print(df_train.flag.value_counts())

0    22743
1     2455
Name: flag, dtype: int64


In [8]:
print(df_transactions.isnull().sum())

print('b316' in train_list)
print('b316' in test_list)

display(df_transactions[df_transactions.to_account.isnull()])
df_transactions = df_transactions.dropna()

from_account            0
to_account              1
transaction_time_utc    1
value                   1
gas                     1
gas_price               1
dtype: int64
False
False


,from_account,to_account,transaction_time_utc,value,gas,gas_price
874984,b316,NaN,NaN,NaN,NaN,NaN


## Exploratory Data Analysis & Preprocessing

In [9]:
print(df_transactions.shape)
print(df_transactions.from_account.nunique(), df_transactions.to_account.nunique())

(874984, 6)
87798 80227


In [10]:
df_transactions.transaction_time_utc = pd.to_datetime(df_transactions.transaction_time_utc)
df_transactions['transaction_year'] = df_transactions.transaction_time_utc.dt.year
df_transactions['transaction_month'] = df_transactions.transaction_time_utc.dt.month
df_transactions['transaction_day'] = df_transactions.transaction_time_utc.dt.day

In [11]:
df_transactions['gas_price'] = df_transactions['gas_price'].div(1000000000)
df_transactions['gas_fee'] = df_transactions['gas'].mul(df_transactions['gas_price'])
df_transactions['gas_fee'].describe()

count    8.749840e+05
mean     2.977605e+07
std      1.501229e+08
min      0.000000e+00
25%      2.404282e+06
50%      8.550000e+06
75%      2.529370e+07
max      4.085285e+10
Name: gas_fee, dtype: float64

In [12]:
df_transactions['is_token'] = np.where(df_transactions['value']==0, 1, 0)
df_transactions['value_digit'] = df_transactions.value.apply(lambda x: len(x))

In [13]:
display(df_transactions)
display(df_transactions[df_transactions.from_account.isin(fraud_list)])
display(df_transactions[df_transactions.to_account.isin(fraud_list)])

,from_account,to_account,transaction_time_utc,value,gas,gas_price,transaction_year,transaction_month,transaction_day,gas_fee,is_token,value_digit
0,a00996,b31499,2020-05-04 14:54:03,0,72585.0,11.500000,2020,5,4,8.347275e+05,0,1
1,a07890,b31500,2020-05-04 14:55:06,0,54426.0,11.349723,2020,5,4,6.177200e+05,0,1
2,a22857,b31501,2020-05-04 14:55:23,0,200000.0,14.024585,2020,5,4,2.804917e+06,0,1
3,a07890,b31502,2020-05-04 14:55:23,108900000000000000,21000.0,11.349723,2020,5,4,2.383442e+05,0,18
4,a21390,b31501,2020-05-04 14:56:05,0,149999.0,32.000000,2020,5,4,4.799968e+06,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
874979,a12937,b177209,2020-09-09 18:57:49,2000000000000000000,21000.0,101.000000,2020,9,9,2.121000e+06,0,19
874980,a22154,b36904,2020-09-09 18:57:49,0,450000.0,93.730000,2020,9,9,4.217850e+07,0,1
874981,a27216,b156802,2020-09-09 19:15:27,0,350000.0,667.000000,2020,9,9,2.334500e+08,0,1
874982,a15828,b176682,2020-09-09 18:57:54,0,1256108.0,93.730000,2020,9,9,1.177350e+08,0,1


,from_account,to_account,transaction_time_utc,value,gas,gas_price,transaction_year,transaction_month,transaction_day,gas_fee,is_token,value_digit
420,a05997,b31735,2020-05-04 15:35:10,0,210000.0,0.000000,2020,5,4,0.000000e+00,0,1
500,a05997,b31735,2020-05-04 15:57:42,0,210000.0,0.000000,2020,5,4,0.000000e+00,0,1
622,a26604,b31818,2020-05-04 16:29:48,6076665128599282058,21000.0,20.000000,2020,5,4,4.200000e+05,0,19
1236,a15674,b32254,2020-05-04 17:26:13,97718000000000000,21000.0,30.000000,2020,5,4,6.300000e+05,0,17
1448,a13198,b31704,2020-05-04 18:48:44,100000000000000000,279749.0,8.000000,2020,5,4,2.237992e+06,0,18
...,...,...,...,...,...,...,...,...,...,...,...,...
874727,a21865,b57644,2020-09-09 18:45:12,0,238968.0,87.000000,2020,9,9,2.079022e+07,0,1
874844,a21865,b57644,2020-09-09 18:49:39,0,279727.0,102.000000,2020,9,9,2.853215e+07,0,1
874848,a21865,b31501,2020-09-09 18:32:25,0,56197.0,85.714286,2020,9,9,4.816886e+06,0,1
874908,a21865,b31501,2020-09-09 18:52:58,0,56221.0,85.714286,2020,9,9,4.818943e+06,0,1


,from_account,to_account,transaction_time_utc,value,gas,gas_price,transaction_year,transaction_month,transaction_day,gas_fee,is_token,value_digit
51,b31530,a10789,2020-05-04 15:06:43,10000000000000000,21000.0,12.000000,2020,5,4,2.520000e+05,0,17
157,b31600,a10789,2020-05-04 14:52:36,10000000000000000,21000.0,12.000000,2020,5,4,2.520000e+05,0,17
163,b31604,a04046,2020-05-04 14:06:35,1087000000000000000,200000.0,12.000000,2020,5,4,2.400000e+06,0,19
459,b31754,a26604,2020-05-04 15:51:28,1999727000000000000,21000.0,13.000000,2020,5,4,2.730000e+05,0,19
472,b31762,a22356,2020-05-04 15:54:08,100000000000000000,21000.0,8.000000,2020,5,4,1.680000e+05,0,18
...,...,...,...,...,...,...,...,...,...,...,...,...
874901,b177186,a03013,2020-09-09 19:10:09,0,66118.0,84.700000,2020,9,9,5.600195e+06,0,1
874923,b168093,a03013,2020-09-09 18:53:42,0,43600.0,87.000000,2020,9,9,3.793200e+06,0,1
874934,b171164,a03013,2020-09-09 18:53:52,0,66118.0,87.000000,2020,9,9,5.752266e+06,0,1
874936,b177200,a03013,2020-09-09 18:36:26,0,48816.0,84.000001,2020,9,9,4.100544e+06,0,1


In [14]:
print(df_transactions.value.value_counts())

0                      611267
5000000000000000        10939
1000000000000000000      7580
7000000000000000         7036
20000000000000000        4896
                        ...  
50188708000000000           1
1360140000000000            1
5057325114380703902         1
43757930000000000           1
37852270545577000           1
Name: value, Length: 142607, dtype: int64


In [15]:
print(df_transactions[df_transactions.from_account.isin(fraud_list)].to_account.value_counts())

b57644     507
b31501     237
b163413    211
b35651     205
b33252      94
          ... 
b90016       1
b89839       1
b89313       1
b89227       1
b177190      1
Name: to_account, Length: 1493, dtype: int64


In [16]:
print(df_transactions[df_transactions.from_account.isin(fraud_list)].value.value_counts())

0                       1893
50000000000000000000     215
1000000000000000000       62
100000000000000000        57
40000000000000000000      25
                        ... 
689761740000000000         1
89533530307100009          1
18611501525781110          1
1500308499923434000        1
36000000000000000000       1
Name: value, Length: 1399, dtype: int64


In [17]:
print(df_transactions[df_transactions.from_account.isin(fraud_list)].gas.value_counts())


21000.0     1830
60000.0      121
77137.0      113
77028.0       56
76746.0       53
            ... 
167433.0       1
56000.0        1
45000.0        1
152561.0       1
279727.0       1
Name: gas, Length: 920, dtype: int64


In [18]:
print(df_transactions[df_transactions.from_account.isin(fraud_list)].gas_price.value_counts())

100.000000    85
480.000000    56
20.000000     52
120.000000    48
35.000000     45
              ..
109.000001     1
71.250000      1
83.124343      1
129.780000     1
26.600000      1
Name: gas_price, Length: 1066, dtype: int64


In [19]:
df_train

,account,flag
0,a17249,0
1,a03683,1
2,a22146,0
3,a26056,1
4,a13971,0
...,...,...
25193,a24443,0
25194,a12337,0
25195,a08122,0
25196,a27826,1
